In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import difflib as dfb
pd.set_option('display.max_columns', None)

Ideally, I would integrate that directly to the demographics file > need to intergate the demographics data import to the DB
Also issue with data for 5 10 lacking 

# Get the data demographics 

In [2]:
admin3_path = '../Data/Mapping layout/Admin3/uga_admbnda_adm3_UBOS_v5_cleaned [CB].shp'
disability_path = '../Data/Disability/Disability_data.csv'
population_path = '../Data/Results/Total_Age_gender.csv'

admin3_df = gpd.read_file(admin3_path)
disability_df = pd.read_csv(disability_path)
pop_df = pd.read_csv(population_path)

# Clean disability data 

8809

In [3]:
disability_df.set_index(['District','Subcounty','Parish'],drop=True, inplace=True)
disability_df.head()

18-30 Years With Disability  \
District  Subcounty              Parish                                     
KALANGALA BUJUMBA                BUJUMBA                               23   
                                 BUNYAMA                              139   
                                 BWENDERO                             162   
                                 MULABANA                              72   
          KALANGALA TOWN COUNCIL KALANGALA A                           18   

                                              18-30 Years With Disability - Percent  \
District  Subcounty              Parish                                               
KALANGALA BUJUMBA                BUJUMBA                                        3.9   
                                 BUNYAMA                                       18.9   
                                 BWENDERO                                      11.3   
                                 MULABANA                                       7.0   
          KALANGALA TOWN COUNCIL KALANGALA A                                    5.5   

                                              2 Years+ With Disability  \
District  Subcounty              Parish                                  
KALANGALA BUJUMBA                BUJUMBA                           142   
                                 BUNYAMA                           432   
                                 BWENDERO                          492   
                                 MULABANA                          267   
          KALANGALA TOWN COUNCIL KALANGALA A                       111   

                                              2 Years+ With Disability - Percent  \
District  Subcounty              Parish                                            
KALANGALA BUJUMBA                BUJUMBA                                     7.7   
                                 BUNYAMA                                    20.7   
                                 BWENDERO                                   14.2   
                                 MULABANA                                    9.7   
          KALANGALA TOWN COUNCIL KALANGALA A                                10.2   

                                              2 Years+ with Hearing Disability  \
District  Subcounty              Parish                                          
KALANGALA BUJUMBA                BUJUMBA                                    35   
                                 BUNYAMA                                    85   
                                 BWENDERO                                   85   
                                 MULABANA                                   62   
          KALANGALA TOWN COUNCIL KALANGALA A                                29   

                                              2 Years+ with Hearing Disability - Percent  \
District  Subcounty              Parish                                                    
KALANGALA BUJUMBA                BUJUMBA                                             1.9   
                                 BUNYAMA                                             4.1   
                                 BWENDERO                                            2.5   
                                 MULABANA                                            2.2   
          KALANGALA TOWN COUNCIL KALANGALA A                                         2.7   

                                              2 Years+ with Remembering Disability  \
District  Subcounty              Parish                                              
KALANGALA BUJUMBA                BUJUMBA                                        38   
                                 BUNYAMA                                       205   
                                 BWENDERO                                      231   
                                 MULABANA                                       89   
          KALANGALA TOWN COUNCIL KALANGALA A        

In [4]:
c = disability_df.groupby(['District','Subcounty'])[['2-17 Years With Disability',
                                                     '18-30 Years With Disability',
                                                     '60 Years+ With Disability']].sum()

parish_perc = disability_df[['2-17 Years With Disability',
                             '18-30 Years With Disability',
                             '60 Years+ With Disability']] / c

parish_perc.rename({'2-17 Years With Disability' : '2_17_perc',
                    '18-30 Years With Disability' : '18_30_perc',
                    '60 Years+ With Disability' : '60_perc'},
                   axis=1,inplace=True)

parish_perc.head()

2_17_perc  18_30_perc   60_perc
District Subcounty         Parish                                    
ABIM     ABIM              ANINATA     0.156962    0.164420  0.079096
                           AREMBWOLA   0.154430    0.169811  0.209040
                           ATUNGA      0.379747    0.455526  0.494350
                           KANU        0.308861    0.210243  0.217514
         ABIM TOWN COUNCIL KALAKALA    0.114669    0.148064  0.126027

In [5]:
disability_df_perc=disability_df.join(parish_perc)
disability_df_perc['2_17_wgtd']=disability_df_perc['2-17 Years With Disability - Percent']*disability_df_perc['2_17_perc']*(1/100)
disability_df_perc['18_30_wgtd']=disability_df_perc['18-30 Years With Disability - Percent']*disability_df_perc['18_30_perc']*(1/100)
disability_df_perc['60_wgtd']=disability_df_perc['60 Years+ With Disability - Percent']*disability_df_perc['60_perc']*(1/100)

In [6]:
disability_sc_df=disability_df_perc.groupby(['District','Subcounty'])[['2_17_wgtd','18_30_wgtd','60_wgtd']].sum()
disability_sc_df.head()

2_17_wgtd  18_30_wgtd   60_wgtd
District Subcounty                                         
ABIM     ABIM                0.124190    0.140202  0.735915
         ABIM TOWN COUNCIL   0.119846    0.114941  0.682521
         ALEREK              0.152905    0.157545  0.768183
         LOTUKEI             0.135073    0.161746  0.715863
         MORULEM             0.123657    0.158467  0.683512

In [34]:
print("Note that disability data is only available for some subcounties, with more than "+
      str(len(admin3_df)-len(disability_sc_df)) +
      " missing data points (about "+
     str(round((len(admin3_df)-len(disability_sc_df))/len(admin3_df),1))+
     " % of the total number of subcounties")

Note that disability data is only available for some subcounties, with more than 204 missing data points (about 0.14186369958275383 % of the total number of subcounties


# Fuzzy matching

In [7]:
disability_sc_df.reset_index(inplace=True)
disability_sc_df['code']=disability_sc_df['District'].apply(lambda x: x[0:5])+"/"+disability_sc_df['Subcounty']
admin3_df['code']=admin3_df.ADM1_EN.apply(lambda x: x[0:5])+"/"+admin3_df['ADM3_EN']

In [8]:
admin3_df['code']

0                       ABIM/ABIM
1          ABIM/ABIM TOWN COUNCIL
2                     ABIM/ALEREK
3                    ABIM/LOTUKEI
4                    ABIM/MORULEM
                  ...            
1433                 ZOMBO/PAIDHA
1434    ZOMBO/PAIDHA TOWN COUNCIL
1435                   ZOMBO/WARR
1436                    ZOMBO/ZEU
1437     ZOMBO/ZOMBO TOWN COUNCIL
Name: code, Length: 1438, dtype: object

In [9]:
codes = []
certainty = []

for code in disability_sc_df['code']:
    match_code = dfb.get_close_matches(code, admin3_df['code'],n=1,cutoff=0.75)
    if len(match_code) > 0:
        codes.append(match_code[0])
        score = dfb.SequenceMatcher(None, code, match_code[0]).ratio()
        certainty.append(score)
    else :
        codes.append(None)
        certainty.append(None)

disability_sc_df['match_code'] = codes
disability_sc_df['certainty'] = certainty

print('This gives us a percentage of null of ' + str(round((disability_sc_df['match_code'].isnull().sum()/len(disability_sc_df))*100,2)) + " %",
     '\nFor a total number of null values of ' + str(disability_sc_df['match_code'].isnull().sum()))

This gives us a percentage of null of 1.87 % 
For a total number of null values of 23


In [10]:
admin3_df.set_index('code',inplace=True)
disability_sc_df.set_index('code',inplace=True,drop=True)
disability_sc_df=disability_sc_df.join(admin3_df['ADM3_PCODE'])

In [11]:
disability_sc_df.head()

,District,Subcounty,2_17_wgtd,18_30_wgtd,60_wgtd,match_code,certainty,ADM3_PCODE
code,,,,,,,,
ABIM/ABIM,ABIM,ABIM,0.124190,0.140202,0.735915,ABIM/ABIM,1.0,UG314101
ABIM/ABIM TOWN COUNCIL,ABIM,ABIM TOWN COUNCIL,0.119846,0.114941,0.682521,ABIM/ABIM TOWN COUNCIL,1.0,UG314102
ABIM/ALEREK,ABIM,ALEREK,0.152905,0.157545,0.768183,ABIM/ALEREK,1.0,UG314103
ABIM/LOTUKEI,ABIM,LOTUKEI,0.135073,0.161746,0.715863,ABIM/LOTUKEI,1.0,UG314104
ABIM/MORULEM,ABIM,MORULEM,0.123657,0.158467,0.683512,ABIM/MORULEM,1.0,UG314105


# Calculate total percentge using demo data 

In [19]:
pop_df.set_index('ADM3_PCODE',inplace=True,drop=True)

In [24]:
pop_df.head()

,uga_f_0,uga_f_1,uga_f_5,uga_f_10,uga_f_15,uga_f_20,uga_f_25,uga_f_30,uga_f_35,uga_f_40,uga_f_45,uga_f_50,uga_f_55,uga_f_60,uga_f_65,uga_f_70,uga_f_75,uga_f_80,uga_m_0,uga_m_1,uga_m_5,uga_m_10,uga_m_15,uga_m_20,uga_m_25,uga_m_30,uga_m_35,uga_m_40,uga_m_45,uga_m_50,uga_m_55,uga_m_60,uga_m_65,uga_m_70,uga_m_75,uga_m_80,total,0_15,20_55,60
ADM3_PCODE,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
UG314101,340.520325,1267.132690,1285.483521,1113.528320,995.159546,992.397949,689.138123,516.849731,374.777740,373.867004,259.933136,208.410614,131.148666,110.434952,40.917202,59.198120,30.671986,27.781048,247.180344,1279.897583,1418.631714,1153.303101,951.640747,884.341431,656.058228,617.542114,414.377930,286.589661,316.741028,188.562927,148.833679,82.237427,49.672493,31.731512,44.615776,31.403730,17620.712097,10052.477890,7059.569962,508.664246
UG314102,489.007599,1819.602539,1845.959229,1599.138428,1429.014160,1425.102539,989.645020,742.170776,538.104370,536.952515,373.234619,299.308228,188.309906,158.583679,58.731026,85.031609,44.056175,39.908520,354.855072,1837.936035,2037.227295,1656.250610,1366.464722,1269.989258,942.065857,886.870728,595.019287,411.496338,454.879700,270.802673,213.757187,118.094223,71.316246,45.552650,64.081718,45.113976,25303.634510,14435.455688,10137.709000,730.469822
UG314103,503.672791,1874.664062,1902.263550,1647.699707,1472.495728,1468.366821,1019.846191,764.775574,554.575745,553.168030,384.520477,308.296417,194.000092,163.310837,60.518482,87.538818,45.360428,41.093136,365.626984,1893.678955,2099.164062,1706.585938,1408.257080,1308.479126,970.745239,913.766968,613.152710,424.189758,468.685852,279.009644,220.226715,121.677261,73.471153,46.945175,65.982437,46.459652,26072.271595,14874.108856,10445.805359,752.357380
UG314104,649.905273,2412.704834,2449.585938,2118.995850,1894.426270,1885.106201,1308.774414,985.160645,717.804932,711.931641,496.318634,396.101746,249.081390,209.773361,78.986481,112.372421,58.280487,52.538551,473.376648,2433.503906,2692.245850,2193.372070,1807.204102,1677.082397,1246.525146,1172.432983,790.103394,547.834717,603.853699,358.340027,284.512299,157.107239,95.706024,60.750694,84.632347,59.371128,33525.803738,19125.320740,13430.964264,969.518734
UG314105,637.564575,2372.915771,2407.326416,2085.395020,1863.660278,1858.570923,1290.737305,968.057861,701.892456,700.327332,486.784973,390.313446,245.763718,206.777313,76.595566,110.847511,57.443878,52.040047,462.749878,2396.675293,2656.591064,2159.926758,1782.026733,1656.226196,1228.593018,1156.567261,776.047241,536.782776,593.184021,353.167419,278.862946,154.040375,93.026375,59.413765,83.544525,58.834564,32999.274597,18824.831787,13221.878891,952.563919


In [23]:
pop_df['total']=0

pop_df['total']=pop_df.sum(axis=1)

pop_df['0_15'] = pop_df[['uga_f_0','uga_f_1','uga_f_5', 'uga_f_10','uga_f_15',
                         'uga_m_0', 'uga_m_1','uga_m_5','uga_m_10','uga_m_15']].sum(axis=1)

pop_df['20_55'] = pop_df[['uga_f_20','uga_f_25','uga_f_30','uga_f_35',
                          'uga_f_40', 'uga_f_45','uga_f_50','uga_f_55',
                          'uga_m_20','uga_m_25', 'uga_m_30','uga_m_35', 
                          'uga_m_40', 'uga_m_45','uga_m_50','uga_m_55']].sum(axis=1)

pop_df['60'] = pop_df[['uga_f_60','uga_f_65', 'uga_f_70','uga_f_75','uga_f_80',
                       'uga_m_60', 'uga_m_65','uga_m_70','uga_m_75','uga_m_80']].sum(axis=1)


In [25]:
disability_map=pop_df.merge(disability_sc_df,left_on='ADM3_PCODE',right_on='ADM3_PCODE',how='left')

In [26]:
disability_map['disability_perc']=(((disability_map['2_17_wgtd']*disability_map['0_15'])
                                   + (disability_map['18_30_wgtd']*disability_map['20_55'])
                                   + (disability_map['60_wgtd']*disability_map['60']))
                                   / disability_map['total'])

In [27]:
disability_map

,ADM3_PCODE,uga_f_0,uga_f_1,uga_f_5,uga_f_10,uga_f_15,uga_f_20,uga_f_25,uga_f_30,uga_f_35,uga_f_40,uga_f_45,uga_f_50,uga_f_55,uga_f_60,uga_f_65,uga_f_70,uga_f_75,uga_f_80,uga_m_0,uga_m_1,uga_m_5,uga_m_10,uga_m_15,uga_m_20,uga_m_25,uga_m_30,uga_m_35,uga_m_40,uga_m_45,uga_m_50,uga_m_55,uga_m_60,uga_m_65,uga_m_70,uga_m_75,uga_m_80,total,0_15,20_55,60,District,Subcounty,2_17_wgtd,18_30_wgtd,60_wgtd,match_code,certainty,disability_perc
0,UG314101,340.520325,1267.132690,1285.483521,1113.528320,995.159546,992.397949,689.138123,516.849731,374.777740,373.867004,259.933136,208.410614,131.148666,110.434952,40.917202,59.198120,30.671986,27.781048,247.180344,1279.897583,1418.631714,1153.303101,951.640747,884.341431,656.058228,617.542114,414.377930,286.589661,316.741028,188.562927,148.833679,82.237427,49.672493,31.731512,44.615776,31.403730,17620.712097,10052.477890,7059.569962,508.664246,ABIM,ABIM,0.124190,0.140202,0.735915,ABIM/ABIM,1.0,0.148264
1,UG314102,489.007599,1819.602539,1845.959229,1599.138428,1429.014160,1425.102539,989.645020,742.170776,538.104370,536.952515,373.234619,299.308228,188.309906,158.583679,58.731026,85.031609,44.056175,39.908520,354.855072,1837.936035,2037.227295,1656.250610,1366.464722,1269.989258,942.065857,886.870728,595.019287,411.496338,454.879700,270.802673,213.757187,118.094223,71.316246,45.552650,64.081718,45.113976,25303.634510,14435.455688,10137.709000,730.469822,ABIM,ABIM TOWN COUNCIL,0.119846,0.114941,0.682521,ABIM/ABIM TOWN COUNCIL,1.0,0.134124
2,UG314103,503.672791,1874.664062,1902.263550,1647.699707,1472.495728,1468.366821,1019.846191,764.775574,554.575745,553.168030,384.520477,308.296417,194.000092,163.310837,60.518482,87.538818,45.360428,41.093136,365.626984,1893.678955,2099.164062,1706.585938,1408.257080,1308.479126,970.745239,913.766968,613.152710,424.189758,468.685852,279.009644,220.226715,121.677261,73.471153,46.945175,65.982437,46.459652,26072.271595,14874.108856,10445.805359,752.357380,ABIM,ALEREK,0.152905,0.157545,0.768183,ABIM/ALEREK,1.0,0.172519
3,UG314104,649.905273,2412.704834,2449.585938,2118.995850,1894.426270,1885.106201,1308.774414,985.160645,717.804932,711.931641,496.318634,396.101746,249.081390,209.773361,78.986481,112.372421,58.280487,52.538551,473.376648,2433.503906,2692.245850,2193.372070,1807.204102,1677.082397,1246.525146,1172.432983,790.103394,547.834717,603.853699,358.340027,284.512299,157.107239,95.706024,60.750694,84.632347,59.371128,33525.803738,19125.320740,13430.964264,969.518734,ABIM,LOTUKEI,0.135073,0.161746,0.715863,ABIM/LOTUKEI,1.0,0.162554
4,UG314105,637.564575,2372.915771,2407.326416,2085.395020,1863.660278,1858.570923,1290.737305,968.057861,701.892456,700.327332,486.784973,390.313446,245.763718,206.777313,76.595566,110.847511,57.443878,52.040047,462.749878,2396.675293,2656.591064,2159.926758,1782.026733,1656.226196,1228.593018,1156.567261,776.047241,536.782776,593.184021,353.167419,278.862946,154.040375,93.026375,59.413765,83.544525,58.834564,32999.274597,18824.831787,13221.878891,952.563919,ABIM,MORULEM,0.123657,0.158467,0.683512,ABIM/MORULEM,1.0,0.153766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1433,UG330119,390.659882,1336.599121,1434.698120,1178.848267,978.816650,828.619690,666.810059,533.539612,522.632202,351.152283,359.776703,259.401001,191.891968,131.065399,91.131294,56.199421,38.471985,17.086094,341.466675,1417.220581,1434.602661,1183.932129,955.633057,665.844299,579.723633,498.301208,454.259552,346.388611,262.773315,203.018753,146.526337,125.583572,88.244614,49.966568,43.917049,20.243830,18185.046194,10652.477142,6870.659225,661.909826,ZOMBO,PAIDHA,0.104534,0.148225,0.725171,ZOMBO/PAIDHA,1.0,0.143632
1434,UG330120,876.082275,2997.418213,3216.708252,2643.403320,2194.728271,1858.279419,1494.814209,1196.154663,1172.103882,787.017090,807.005249,581.951233,430.393829,293.981628,204.255524,126.071823,86.2

In [28]:
disability_map.to_csv('../Data/Results/Age_gender_disability.csv', index=False)